In [11]:
%run ../../../ingestion/ingestion.ipynb
%run ../../../data\ quality/quality.ipynb
%run ../../../write/write.ipynb
%run ../../../alerts/webhook.ipynb

25/05/07 22:20:07 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/05/07 22:20:07 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [12]:
from pyspark.sql import functions as F

In [13]:
# Cria uma instância da classe WebhookTeams utilizando o arquivo de configuração localizado em ../../config/teams.json
webhook = WebhookTeams("../../../config/teams.json")

In [14]:
try:
    # Cria um leitor de CSV com delimitador de linha definido como ";"
    reader = CSVReader(line_delimiter=";")
    
    # Executa a leitura do arquivo CSV e retorna um DataFrame
    df = client_code(reader, "../../../layer/bronze/economia/animais_cabecas.csv")
except:
    # Em caso de erro durante a leitura, envia uma notificação via Teams
    webhook.send_teams_notification("Erro de leitura!")


In [15]:
df.show()

+-------------------+---------------+---------+----+
|Codigo do municipio|Animais_cabecas|Cod_Gecon|Data|
+-------------------+---------------+---------+----+
|            3500105|          34998|      101|2019|
|            3500204|           5987|      101|2019|
|            3500303|           4085|      101|2019|
|            3500402|          10200|      101|2019|
|            3500501|           5401|      101|2019|
|            3500550|          10021|      101|2019|
|            3500709|          40000|      101|2019|
|            3500758|           6538|      101|2019|
|            3500808|          14000|      101|2019|
|            3500907|          25734|      101|2019|
|            3501004|          14900|      101|2019|
|            3501103|          24775|      101|2019|
|            3501152|            880|      101|2019|
|            3501202|          42310|      101|2019|
|            3501301|          45137|      101|2019|
|            3501400|          13379|      101

In [16]:
try:
    # Cria um leitor de CSV com delimitador de linha definido como ","
    reader = ParquetReader()
    
    # Executa a leitura do arquivo CSV e retorna um DataFrame
    df_din = client_code(reader, "../../../layer/prata/dimensao/dimensao_ibge/*.parquet")
    df_din = df_din.select("cod_ibge", "municipios", "reg_administrativa")
except:
    # Em caso de erro durante a leitura, envia uma notificação via Teams
    webhook.send_teams_notification("Erro de leitura!")


In [17]:
# Instancia a classe de utilitários de qualidade de dados com o DataFrame original
dq = DataQualityUtils(df)

# Encadeia as transformações de qualidade de dados no DataFrame
df_tratado = (
    dq.remove_espacos_em_colunas()  # Remove espaços extras dos nomes das colunas
      .trim_strings()  # Remove espaços em branco no início e fim das strings de todas as colunas
      #.remover_caracteres_especiais(['localidades'])  # (Comentado) Remove caracteres especiais da coluna 'localidades'
      .remover_duplicatas()  # Remove registros duplicados do DataFrame
      .preencher_nulos({  # Preenche valores nulos com zero nas colunas especificadas
          'Animais_cabecas': 0
      })
      .converter_para_numerico(['Animais_cabecas'])
      .get_df()  # Retorna o DataFrame tratado
)


In [18]:
df_din.show()

+--------+--------------------+--------------------+
|cod_ibge|          municipios|  reg_administrativa|
+--------+--------------------+--------------------+
| 3512704|          Corumbataí|      RA de Campinas|
| 3538204|         Pinhalzinho|      RA de Campinas|
| 3549409|São Joaquim da Barra|        RA de Franca|
| 3528205|           Macedônia|RA de São José do...|
| 3501905|              Amparo|      RA de Campinas|
| 3527702|           Luiziânia|     RA de Araçatuba|
| 3550803|São Sebastião da ...|      RA de Campinas|
| 3556008|              Urupês|RA de São José do...|
| 3556453|Vargem Grande Pau...|    RM  de São Paulo|
| 3525607|        João Ramalho|       RA de Marília|
| 3537701|             Piacatu|     RA de Araçatuba|
| 3555109|       Tupi Paulista|RA de Presidente ...|
| 3520608|             Indiana|RA de Presidente ...|
| 3522109|            Itanhaém|        RA de Santos|
| 3524303|         Jaboticabal|RA de Ribeirão Preto|
| 3526605|           Lavrinhas|RA de São José 

In [19]:
df_tratado.show()

+-------------------+---------------+---------+----+
|Codigo_do_municipio|Animais_cabecas|Cod_Gecon|Data|
+-------------------+---------------+---------+----+
|            3516200|        21614.0|      101|2019|
|            3505104|         9427.0|      101|2020|
|            3540606|        22700.0|      101|2020|
|            3545159|         6062.0|      101|2020|
|            3556107|        13200.0|      101|2020|
|            3534807|        18750.0|      101|2013|
|            3512605|        17000.0|      101|2022|
|            3527256|         9939.0|      101|2022|
|            3530201|       132290.0|      101|2022|
|            3532900|          646.0|      101|2022|
|            3535705|         1300.0|      101|2022|
|            3552007|        14220.0|      101|2018|
|            3531704|        13494.0|      101|2017|
|            3531803|        10400.0|      101|2017|
|            3520707|        27001.0|      101|2021|
|            3523701|        11600.0|      101

In [20]:
# Faz o relacionamento com a dimensão de códigos do IBGE
df_tratado = df_tratado.alias("tratado").join(
    df_din.alias("din"),
    F.col("tratado.Codigo_do_municipio") == F.col("din.cod_ibge"),
    how="left"
)

In [21]:
# Exibe as primeiras linhas do DataFrame tratado para visualização dos dados após as transformações
df_tratado.show()

+-------------------+---------------+---------+----+--------+--------------------+--------------------+
|Codigo_do_municipio|Animais_cabecas|Cod_Gecon|Data|cod_ibge|          municipios|  reg_administrativa|
+-------------------+---------------+---------+----+--------+--------------------+--------------------+
|            3516200|        21614.0|      101|2019| 3516200|              Franca|        RA de Franca|
|            3505104|         9427.0|      101|2020| 3505104|             Barbosa|     RA de Araçatuba|
|            3540606|        22700.0|      101|2020| 3540606|         Porto Feliz|      RA de Sorocaba|
|            3545159|         6062.0|      101|2020| 3545159|            Saltinho|      RA de Campinas|
|            3556107|        13200.0|      101|2020| 3556107|     Valentim Gentil|RA de São José do...|
|            3534807|        18750.0|      101|2013| 3534807|          Ouro Verde|RA de Presidente ...|
|            3512605|        17000.0|      101|2022| 3512605|   

In [22]:
try:
    # Cria uma instância da classe responsável por exportar em formato Parquet
    exporter = ParquetExport()

    # Exporta o DataFrame para o caminho local especificado
    exporter.export(df_tratado, "../../../layer/prata/economia/animais_cabecas/", destination="local")
except:
    # Em caso de erro durante a exportação, envia uma notificação via Teams
    webhook.send_teams_notification("Erro de escrita!")
